In [ ]:
!pip install -q spotipy

In [ ]:
from IPython.display import Audio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_secrets import UserSecretsClient

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import SpotifyException

In [ ]:
%matplotlib inline

In [ ]:
user_secrets = UserSecretsClient()
spotify_cid = user_secrets.get_secret("spotify_cid")
spotify_secret = user_secrets.get_secret("spotify_secret")
client_credentials_manager = SpotifyClientCredentials(client_id=spotify_cid, client_secret=spotify_secret)
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
spotify.max_retries=10
spotify.backoff_factor = 0.4
spotify.retries = 10

In [ ]:
df = pd.read_csv('/kaggle/input/muse-the-musical-sentiment-dataset/muse_v3.csv').astype({'valence_tags': np.float32, 'arousal_tags':np.float32, 'dominance_tags':np.float32})

In [ ]:
random_sample = df.dropna(subset=['spotify_id']).sample(1)
track = random_sample['track'].to_list()[0]
artist = random_sample['artist'].to_list()[0]
genre = random_sample['genre'].to_list()[0]
spotify_id = random_sample['spotify_id'].to_list()[0]
print(f'artist: {artist}\ntrack: {track}\ngenre: {genre}\nspotify_id: {spotify_id}')

In [ ]:
spotify_track['preview_url']

In [ ]:
spotify_track  = spotify.track(spotify_id)
print(spotify_track.keys(), '\n')
low_level_features = spotify.audio_analysis(spotify_id)
print(list(low_level_features.keys()), '\n')
high_level_features = spotify.audio_features(spotify_id)
print(list(high_level_features[0].keys()), '\n')

In [ ]:
url = spotify_track.get('preview_url')
Audio(url+".mp3") if url else None

In [ ]:
high_level_features

In [ ]:
segments = low_level_features.get("segments", -1)
bars = low_level_features.get("bars", -1)
beats = low_level_features.get("beats", -1)
tatums = low_level_features.get("tatums", -1)
sections = low_level_features.get("sections", -1)
df = pd.DataFrame(segments)
pitches = pd.DataFrame(list(df.pitches.values))
timbre = pd.DataFrame(list(df.timbre.values))

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(30,10))
sns.heatmap(pitches.T, ax=ax[0], xticklabels=False, cbar=False);
sns.heatmap(timbre.T, ax=ax[1], cbar=False, cmap="YlGnBu");
fig.suptitle(f'Audio features for "{track}" by "{artist}"', fontsize=20)
ax[0].set_title('Pitches across time')
ax[1].set_title('Timbres across time');